In [1]:
!rm -rf ~/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-OCR-2*
!rm -rf ~/.cache/huggingface/modules/transformers_modules/deepseek-ai--DeepSeek-OCR-2*

!pip uninstall -y transformers tokenizers flash-attn
!pip uninstall -y torchvision torchaudio
!pip install --no-cache torch==2.6.0 --index-url https://download.pytorch.org/whl/cu118
!pip install --no-cache transformers==4.46.3 tokenizers==0.20.3
!pip install torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118
!pip install einops addict easydict

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 231.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 231.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 223.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 298.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 307.2 MB/s eta 0:00:0

In [2]:
import torch
import torchvision
print(torch.__version__)       # Should be 2.6.0
print(torchvision.__version__) # Should be 0.21.0
import transformers
print(transformers.__version__) # Should be 4.46.3 from earlier fix

2.6.0+cu118
0.21.0+cu118
4.46.3


In [3]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
# ============================================================
# Ячейка 2
# ============================================================

import os, json, logging, re, time, random, gc
import torch
from transformers import AutoModel, AutoTokenizer
from openai import OpenAI
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT

# ======================== НАСТРОЙКИ ========================

API_KEY    = "your-api-key"
BASE_URL   = "https://openrouter.ai/api/v1"
LLM_MODEL  = "xiaomi/mimo-v2-flash"

OCR_MODEL  = "deepseek-ai/DeepSeek-OCR-2"

# 📂 Единственная папка, в которой лежат фотографии документов
INPUT_FOLDER  = "/kaggle/input/6-folder-it-park/6"

OUTPUT_FILE   = "OPIS_PAPKI.docx"
TEMP_DIR      = "temp_ocr"

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"]   = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

logging.basicConfig(level=logging.INFO, format="%(asctime)s  %(message)s")

# ==================== ЗАГРУЗКА OCR-МОДЕЛИ ====================

logging.info("Загрузка DeepSeek-OCR-2 …")
tokenizer = AutoTokenizer.from_pretrained(OCR_MODEL, trust_remote_code=True)
model = AutoModel.from_pretrained(OCR_MODEL, trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)
logging.info("Модель загружена ✓")

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)


# ======================== УТИЛИТЫ ========================

def clean_ocr(text: str) -> str:
    """Убираем служебные токены DeepSeek-OCR."""
    if not text:
        return ""
    text = re.sub(r'<\|ref\|>.*?<\|/ref\|>', '', text)
    text = re.sub(r'<\|det\|>\[\[.*?\]\]<\|/det\|>', '', text)
    text = re.sub(r'<\|.*?\|>', '', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()


def ocr_single_image(image_path: str) -> str:
    """OCR одного изображения → чистый текст."""
    os.makedirs(TEMP_DIR, exist_ok=True)
    result_file = os.path.join(TEMP_DIR, "result.mmd")

    if os.path.exists(result_file):
        os.remove(result_file)

    torch.cuda.empty_cache(); gc.collect()

    prompt = "<image>\n<|grounding|>Convert the document to markdown. "
    model.infer(
        tokenizer,
        prompt=prompt,
        image_file=image_path,
        output_path=TEMP_DIR,
        base_size=1024,
        image_size=768,
        crop_mode=True,
        save_results=True,
    )

    if os.path.exists(result_file):
        with open(result_file, "r", encoding="utf-8") as f:
            return clean_ocr(f.read())
    return ""


def ask_llm(ocr_text: str, max_retries: int = 6) -> dict:
    """
    Отправляем распознанный текст в LLM.
    Получаем JSON с полями:
      name            — наименование документа
      original_copy   — «Оригинал» или «Копия»
      pages           — кол-во листов  (число или "")
      copies          — кол-во экземпляров (число или "")
    """

    system = """Ты — архивариус. Тебе дан OCR-текст ОДНОГО документа (фото).

Определи:
1. **name** — краткое наименование документа
   (например: «Устав ООО "Ромашка"», «Свидетельство о регистрации»,
    «Договор аренды №12 от 01.03.2021», «Доверенность» и т.п.).
2. **original_copy** — это оригинал или копия?
   Признаки копии: штамп «КОПИЯ», «Копия верна», плохое качество ксерокса,
   отсутствие «живой» подписи/печати.  Если не ясно — пиши «Копия».
3. **pages** — сколько листов занимает документ.
   Если на фото виден только 1 лист — ставь 1.
   Если есть нумерация страниц — используй её.
   Если не понятно — ставь 1.
4. **copies** — количество экземпляров. Обычно 1, если не указано иное.

Верни ТОЛЬКО JSON (без ```, без пояснений):
{"name": "...", "original_copy": "...", "pages": ..., "copies": ...}
"""

    truncated = ocr_text[:60000]
    base_delay = 1.5

    for attempt in range(1, max_retries + 1):
        try:
            resp = client.chat.completions.create(
                model=LLM_MODEL,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user",   "content": f"OCR-текст документа:\n\n{truncated}"},
                ],
                temperature=0.1,
                max_tokens=1000,
            )
            raw = resp.choices[0].message.content
            raw = raw.replace("```json", "").replace("```", "").strip()
            return json.loads(raw)
        except Exception as e:
            logging.error(f"LLM attempt {attempt}/{max_retries}: {e}")
            if attempt == max_retries:
                return {"name": "Не распознано", "original_copy": "", "pages": 1, "copies": 1}
            time.sleep(base_delay * 2 ** (attempt - 1) * random.uniform(0.8, 1.2))


# =================== ГЕНЕРАЦИЯ WORD ====================

def build_word(rows: list, output_path: str):
    """
    Создаёт .docx с таблицей:
    № | Наименование документа | Оригинал/копия | Кол-во листов | Кол-во экз.
    """
    doc = Document()

    # --- Заголовок ---
    title = doc.add_paragraph()
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = title.add_run("Опись учредительных и правоустанавливающих документов")
    run.bold = True
    run.font.size = Pt(14)
    run.font.name = "Times New Roman"

    doc.add_paragraph()  # пустая строка

    # --- Таблица ---
    cols = ["№", "Наименование документа", "Оригинал/\nкопия", "Кол-во\nлистов", "Кол-во\nэкз."]
    table = doc.add_table(rows=1 + len(rows), cols=len(cols))
    table.style = "Table Grid"
    table.alignment = WD_TABLE_ALIGNMENT.CENTER

    # ширины колонок (примерно)
    widths = [Cm(1.2), Cm(10), Cm(2.8), Cm(2.2), Cm(2.2)]

    # шапка
    for i, col_name in enumerate(cols):
        cell = table.rows[0].cells[i]
        cell.width = widths[i]
        p = cell.paragraphs[0]
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = p.add_run(col_name)
        run.bold = True
        run.font.size = Pt(11)
        run.font.name = "Times New Roman"

    # данные
    for r_idx, row in enumerate(rows):
        cells = table.rows[r_idx + 1].cells
        values = [
            str(row.get("num", r_idx + 1)),
            str(row.get("name", "")),
            str(row.get("original_copy", "")),
            str(row.get("pages", "")),
            str(row.get("copies", "")),
        ]
        for c_idx, val in enumerate(values):
            cells[c_idx].width = widths[c_idx]
            p = cells[c_idx].paragraphs[0]
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER if c_idx != 1 else WD_ALIGN_PARAGRAPH.LEFT
            run = p.add_run(val)
            run.font.size = Pt(11)
            run.font.name = "Times New Roman"

    doc.save(output_path)
    logging.info(f"Файл сохранён: {output_path}")


# ======================== MAIN ========================

def main():
    if not os.path.isdir(INPUT_FOLDER):
        print(f"❌ Папка не найдена: {INPUT_FOLDER}")
        return

    # Собираем изображения
    exts = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif')
    images = sorted([
        f for f in os.listdir(INPUT_FOLDER)
        if f.lower().endswith(exts) and not f.startswith('.')
    ])

    if not images:
        print("❌ В папке нет изображений!")
        return

    print(f"Найдено изображений: {len(images)}\n")

    rows = []

    for idx, fname in enumerate(images, 1):
        img_path = os.path.join(INPUT_FOLDER, fname)
        logging.info(f"[{idx}/{len(images)}] OCR: {fname}")

        # 1) OCR
        ocr_text = ocr_single_image(img_path)
        if not ocr_text or len(ocr_text) < 10:
            logging.warning(f"  ⚠ Пустой OCR для {fname}, пропускаю")
            continue

        logging.info(f"  OCR получено {len(ocr_text)} символов")

        # (сохраняем для отладки)
        dbg = os.path.join(TEMP_DIR, f"debug_{idx}_{fname}.txt")
        with open(dbg, "w", encoding="utf-8") as f:
            f.write(ocr_text)

        # 2) LLM-анализ
        data = ask_llm(ocr_text)
        logging.info(f"  LLM → {data.get('name', '?')}")

        rows.append({
            "num":           idx,
            "name":          data.get("name", "Не распознано"),
            "original_copy": data.get("original_copy", ""),
            "pages":         data.get("pages", 1),
            "copies":        data.get("copies", 1),
        })

        print(f"  ✓ [{idx}/{len(images)}] {data.get('name', '?')[:70]}")

        torch.cuda.empty_cache(); gc.collect()

    # 3) Собираем Word
    if rows:
        build_word(rows, OUTPUT_FILE)
        print(f"\n{'='*50}")
        print(f"✅  Опись готова: {OUTPUT_FILE}")
        print(f"    Документов в описи: {len(rows)}")
        print(f"{'='*50}")
    else:
        print("⚠ Ни один документ не был распознан.")


main()

2026-02-12 09:05:46,507  Загрузка DeepSeek-OCR-2 …


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 144, 1280])
<|ref|>text<|/ref|><|det|>[[160, 168, 700, 205]]<|/det|>
Мною, Слушайте и уважайте Худа вишне 47

<|ref|>text<|/ref|><|det|>[[160, 201, 592, 220]]<|/det|>
(должность, фамилия, инициалы, место жительства)

<|ref|>text<|/ref|><|det|>[[160, 214, 500, 234]]<|/det|>
опечатанное помещение ГАУ, 22-неделя

<|ref|>text<|/ref|><|det|>[[160, 230, 500, 245]]<|/det|>
(наименование места хранения)

<|ref|>text<|/ref|><|det|>[[159, 238, 860, 440]]<|/det|>
с хранящимися в нем Зарец Сун рефлйон SHT-ST гибкий-2шт
Кресто Горкель, убог гребной-10 шт. Стои рабочей, 10ехто
Х 50 на зра, оперед-10 шт. Ремолька 3 Свои 3из
ЯК телевизоне А К М МЕДИ РТ-STAND-7-1ег;
LED-телевизоне Sharp IC-4ЕЕЛЕФТОРУ-1ег;
Свои Таблей Х-1-1ег; Педагогический-двелетерАуа
Морк 0+1Д-1ег; Светлак ИФОХ 2Д 6Х.5СЕ-6 ед.
Зеегспраевое Крепо-4уег; Торга, догетна Миклотк
Кбе АКБ1-5аерад 9.5бд; 2.4/5дк2-2д.ег;
Лоупирувайор Микрогнек НФ60/GS 5х 10м0/1000 асе-
-ТК SPB РЕЕ + 1

image: 0it [00:00, ?it/s]
other: 100%|██████████| 25/25 [00:00<00:00, 81474.44it/s]
2026-02-12 09:45:45,925    OCR получено 1539 символов
2026-02-12 09:45:47,331  HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-12 09:45:47,752    LLM → Акт приема-передачи имущества на ответственное хранение


  ✓ [23/23] Акт приема-передачи имущества на ответственное хранение


2026-02-12 09:45:48,451  Файл сохранён: OPIS_PAPKI.docx



✅  Опись готова: OPIS_PAPKI.docx
    Документов в описи: 23
